In [ ]:
def getdist(X1, X2):
    # 유클리디안 거리
    from math import sqrt
    if len(X1) != len(X2):
        exit(1)
    return sqrt(sum((X1[i] - X2[i]) ** 2 for i in range(len(X1))))

def KMeans(X, k, medoid=False):
    import random
    import numpy as np
    from pandas import DataFrame

    dt = np.c_[np.array(X), np.zeros(shape=len(X))]
    # 거리기반이므로 정규화
    for i in dt:
        i = (i - np.mean(i)) / np.std(i)
    # 처음엔 랜덤한 데이터 k 개 잡기
    centers = np.array([dt[i][:-1] for i in random.sample(range(len(X)), k)])
    while 1:
        sums = np.zeros_like(centers)
        cnts = [0] * k
        for i in dt:
            mindist, group = 100000000, 0

            # 여기서 가장 가까운 중심점 찾기
            for j in range(k):
                temp = getdist(i[:-1], centers[j])
                if temp < mindist:
                    mindist, group = temp, j
            # 각 그룹당 합과 개수 모두 있어야 하므로 여기서 더해주기
            sums[group] += i[:-1]
            cnts[group] += 1
            i[-1] = group

        # 일반 k means 알고리즘의 경우 이 avg 를 새로운 중심점으로 이용
        new = np.array([sums[i] / cnts[i] if cnts[i] > 0 else centers[i] for i in range(k)])
        # medoid=True 면 k medoid 알고리즘을 이용하기 위해 avg 에서 가장 가까운 점들을 찾기
        if medoid:
            medoids = np.zeros_like(new)
            dists = np.array([100000000] * len(new))
            for i in dt:
                group = int(i[-1])
                tmp = getdist(new[group], i[:-1])
                if dists[group] > tmp:
                    medoids[group] = np.array(i[:-1])
                    dists[group] = tmp
            new = medoids
        # 루프 하나가 끝났으면 출력해주기
        print("one loop done")
        # 중심점에 변화가 없다면 끝
        if np.array_equal(new, centers):
            break
        # 아니면 중심점 바꿔주기
        centers = new
    # 각 군집에 몇개의 데이터들이 들어갔는지 출력해주고
    print([i for i in cnts])
    # 만들어진 변수들과 군집의 최종 중심점을 출력
    # 중심점은 표준화되어있음을 유의해야함
    return dt[:, -1], centers
from sklearn import datasets
from pandas import DataFrame
import numpy as np
iris = datasets.load_iris()
data = DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])
x = data.drop(columns=['target'], axis=1)
data['group'] = KMeans(data.drop(columns=['target'], axis=1), 3, medoid=True)[0]
print('k medoid')
for i in range(3):
    tmp = data[data['target'] == i]
    print(i, [len(tmp[tmp['group'] == j]) for j in range(3)])
del data['group']
data['group'] = KMeans(data.drop(columns=['target'], axis=1), 3)[0]
print('k means')
for i in range(3):
    tmp = data[data['target'] == i]
    print(i, [len(tmp[tmp['group'] == j]) for j in range(3)])
# 몇번 실행해본 결과 실행할 때마다 편차가 꽤나 있고 medoid 를 쓰건 안쓰건 성능차이가 눈에 띄진 않는다.
# 그러나 medoid 가 일반적으로 더 빨리 수렴하는 것으로 보인다.
# 일반적으로 target = 0 인 데이터는 잘 걸러지는 반면 1, 2 인 데이터들은 섞인다.

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
tr = pd.read_csv("data/train.csv")
# 쓸만한 변수를 골라 문자열을 숫자로 바꿔주자.
# 도로가 포장 되어있는지에 따라 값이 영향이 있을 수 있다.
tr['Street'].replace({'Grvl': 0, 'Pave': 1}, inplace=True)
# 골목이 포장되어 있는지에 따라 값이 영향이 있을 수 있고, NA<Grvl<Pave 의 순서로 좋은 영향을 끼친다는 걸 짐작 가능하다.
tr['Alley'].replace({np.nan: 0, 'Grvl': 1, 'Pave': 1}, inplace=True)
# 인프라가 잘 되어있는지에 따라 영향이 있고, 아래 순서대로 좋은 영향을 끼친다.
tr['Utilities'].replace({'ELO': 0, 'NoSeWa': 1, 'NoSewr': 2, 'AllPub': 3}, inplace=True)
# 여러 피쳐에 대해 평가가 되어 있고 이는 값에 영향을 줄 수 있다. 공통으로 쓰여진 문자들을 이용하여 값을 바꿔주기
# 아래 순서로 긍정적인 영향을 끼치므로 이를 숫자로 바꾸자.
strs = [np.nan, 'Po', 'Fa', 'TA', 'Gd', 'Ex']
for i in tr.columns:
    if all(j in strs for j in tr[i].unique()):
        tr[i].replace({np.nan: 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}, inplace=True)
# 차고의 상태를 평가한 지표
tr['GarageFinish'].replace({np.nan: 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}, inplace=True)
# 사유차도가 포장되어있는가 아닌가
tr['PavedDrive'].replace({'N': 0, 'P': 1, 'Y': 2}, inplace=True)
# 펜스의 상태
tr['Fence'].replace({np.nan: 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4}, inplace=True)
# 팔린지 얼마 지났는지 월 단위로 변환 후 연 단위 삭제
from datetime import datetime
tr['MoSold'] = (datetime.now().year - tr['YrSold']) * 12 + (datetime.now().month - tr['MoSold'])
del tr['YrSold']
# 그 외 문자열 변수는 수치화 하기 어려우니 삭제
for i in tr.columns:
    if tr.dtypes[i] == 'object':
        del tr[i]
# 0에 가까움을 알 수 있고 이는 짐작 가능한 결과다.
# 왜냐면 각 집 타입에 대한 값은 임의로 지정해 줬을 뿐 값 자체에 의미가 있는게 아님 그래서 삭제
del tr['MSSubClass']
# 결측치가 생기는 경우는 예를 들어 차고가 없는 상황에서 차고를 만든 연도를 입력해야 할 때 나타난다.
# 이러한 경우는 그냥 0으로 만들어 값이 예측에 영향을 없게 만들어줘도 무방하므로 모두 0으로 만들자.
for i in tr.columns:
    tr[i].replace({np.nan: 0}, inplace=True)
# 대체적으로 상관관계가 낮다는 것을 알 수 있다.
# 0이 너무 많음과 동시에 상관관계가 낮으므로 제거
# 그래도 fence 나 alley 와 같은 변수들은 충분히 값에 영향을 끼칠수 있다고 판단
# 그래서 이들을 삭제 안할 정도인 0.05로 정해서 이 이하면 삭제
for i in tr.columns:
    if len(tr[tr[i] == 0]) > 1000:
        if abs(tr.corr()[i]['SalePrice']) < 0.05:
            del tr[i]
# 차고 설립연도, 질, 상태가 강한 상관관계를 가지고 있고 이는 당연하다.
# 질과 상태는 주관적인 변수지만 연도는 객관적이므로 질과 상태를 삭제하자.
# 넓이와 차를 몇개 넣을 수 있는지도 강한 상관관계를 가지고 있다.
# 넓이가 넓으면 차를 넣는 것 뿐 아니라 다른 용도로 쓰일 가능성도 있다. 그러므로 넓이를 남기고 차를 삭제
del tr['GarageQual']
del tr['GarageCond']
del tr['GarageCars']
# 예상보다 서로 강한 상관관계를 가지고 않다.
# 하지만 당연히 total 은 독립이 아닐것이므로 삭제
# 질과 상태의 경우 상관관계가 없지는 않아보인다.
# 그래서 둘을 모두 고려하기 위해 합으로 바꾸자
del tr['TotalBsmtSF']
tr['Bsmt'] = tr['BsmtQual'] + tr['BsmtCond']
del tr['BsmtQual']
del tr['BsmtCond']
# id 제거
del tr['Id']
# 이 외에도 상관계수가 높은 경우가 어느 경우가 있을지 보자
corrs = tr.corr()
for i in range(len(tr.columns)):
    x = tr.columns[i]
    for j in range(i + 1, len(tr.columns)):
        y = tr.columns[j]
        if corrs[x][y] > 0.7:
            print(x, y, tr.corr()[x][y])
# PoolArea 와 PoolQC 역시 매우 강한 상관관계가 있다. PoolArea 가 더 객관적이므로 PoolQC 삭제
# 벽난로의 수와 그의 질 또 한 매우 상관관계가 있다.
# 이 경우엔 난로 수가 무조건 많다고 좋다는 보장이 없다. 그러므로 Fireplaces 를 삭제
# 지상에 있는 방의 수와 집의 지상 넓이는 넓이를 채택. TotRmsAbv 삭제. 이유는 SalePrice 와의 상관관계가 더 높다.
# 그 외에는 충분히 같이 있을수 있는 변수라 판단
del tr['PoolQC']
del tr['Fireplaces']
del tr['TotRmsAbvGrd']
# 관찰 결과 Street, Alley, Utilities, ExterCond, KitchenAbvGr, BedroomAbvGr, PavedDrive OverallCond 는 특정 값에 치우쳐 있음을 알 수 있다.
for i in tr.columns:
    if i in ['Street', 'Alley', 'Utilities', 'ExterCond', 'KitchenAbvGr', 'BedroomAbvGr', 'PavedDrive']:
        del tr[i]
# Fence Quality 와 OverallCond 는 높을수록 좋아야 하는데 상관계수가 음수라는 것은 가격에 영향을 안준다는 뜻
del tr['Fence']
del tr['OverallCond']
# train test 를 7:3 으로 나누고 앙상블 알고리즘 중 하나인 xgboost regressor 를 이용합니다.
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
tr['SalePrice'].astype('float64')
train, test = train_test_split(tr, test_size=0.3, random_state=0)
X_tr = train.drop(columns=['SalePrice'], axis=1)
y_tr = train['SalePrice']
X_tst, y_tst = test.drop(columns=['SalePrice'], axis=1), test['SalePrice']
model_xgb = XGBRegressor()
res = model_xgb.fit(X_tr, y_tr)
# 평가기준은 kaggle 에 있는 rmsle 값을 이용. 출처 https://www.kaggle.com/marknagelberg/rmsle-function
# .133 가량 나옴
def rmsle(y, y_pred):
    from math import log
    terms_to_sum = [(log(y_pred[i] + 1) - log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5
# 내가 만든 k means 써보자
def getdist(X1, X2):
    from math import sqrt
    if len(X1) != len(X2):
        exit(1)
    return sqrt(sum((X1[i] - X2[i]) ** 2 for i in range(len(X1))))

def KMeans(X, k):
    import random
    import numpy as np
    from pandas import DataFrame

    X['setnum'] = 0
    dt = np.array(X)
    for i in dt:
        i = (i - np.mean(i)) / np.std(i)
    centers = np.array([dt[i][:-1] for i in random.sample(range(len(X)), k)])
    while 1:
        sums = np.zeros_like(centers)
        cnts = [0] * k
        for i in dt:
            mindist, group = 100000000, 0

            for j in range(k):
                temp = getdist(i[:-1], centers[j])
                if temp < mindist:
                    mindist, group = temp, j

            sums[group] += i[:-1]
            cnts[group] += 1
            i[-1] = group


        avg = np.array([sums[i] / cnts[i] if cnts[i] > 0 else np.zeros_like(sums[i]) for i in range(k)])
        medoids = np.zeros_like(avg)
        dists = np.array([100000000] * len(avg))
        for i in dt:
            group = int(i[-1])
            tmp = getdist(avg[group], i[:-1])
            if dists[group] > tmp:
                medoids[group] = np.array(i[:-1])
                dists[group] = tmp

        print("one loop done")
        if np.array_equal(medoids, centers):
            break
        centers = np.array(medoids)
    print([i for i in cnts])

    X['setnum'] = dt[:, -1]
    return centers
centers = KMeans(X_tr, 5)
# 트레이닝 데이터로 구한 centers 를 이용하여 kmeans 변수 만들기
tmp = np.array(X_tst)
group = []
for i in tmp:
    i = (i - np.mean(i)) / np.std(i)
for i in tmp:
    dist, g = 100000000, 0
    for j in range(5):
        temp = getdist(i, centers[j])
        if temp < dist:
            dist, g = temp, j
    group.append(j)
# 성능이 좋아졌으므로 채택.
# 이제 test.csv 에 kmeans 변수도 넣고 train 데이터에 없는 피쳐를 제거.
test = pd.read_csv('data/test.csv')
sub = DataFrame(data=test['Id'])
test['Street'].replace({'Grvl': 0, 'Pave': 1}, inplace=True)
test['Alley'].replace({np.nan: 0, 'Grvl': 1, 'Pave': 1}, inplace=True)
test['Utilities'].replace({'ELO': 0, 'NoSeWa': 1, 'NoSewr': 2, 'AllPub': 3}, inplace=True)
strs = [np.nan, 'Po', 'Fa', 'TA', 'Gd', 'Ex']
for i in test.columns:
    if all(j in strs for j in test[i].unique()):
        test[i].replace({np.nan: 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}, inplace=True)
test['GarageFinish'].replace({np.nan: 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}, inplace=True)
test['PavedDrive'].replace({'N': 0, 'P': 1, 'Y': 2}, inplace=True)
test['Fence'].replace({np.nan: 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4}, inplace=True)
from datetime import datetime
test['MoSold'] = (datetime.now().year - test['YrSold']) * 12 + (datetime.now().month - test['MoSold'])
del test['YrSold']
test['Bsmt'] = test['BsmtQual'] + test['BsmtCond']
for i in test.columns:
    if i in X_tr.columns:
        continue
    del test[i]
tmp = np.array(test)
group = []
for i in tmp:
    i = (i - np.mean(i)) / np.std(i)
for i in tmp:
    dist, g = 100000000, 0
    for j in range(5):
        temp = getdist(i, centers[j])
        if temp < dist:
            dist, g = temp, j
    group.append(j)
test['setnum'] = np.array(group)
model_xgb = XGBRegressor(max_depth=4, learning_rate=0.05, subsample=0.73, colsample_bytree=0.56, nthread=-1)
a = model_xgb.fit(X_tr, y_tr)
sub['SalePrice'] = np.array(a.predict(test))
# 제출
sub.to_csv('submission.csv', index=False)

0.1272 -> 0.1266으로 좋아짐